In [1]:
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import Dataset, DataLoader
torch.set_printoptions(precision=2)

class CsvDataset(Dataset):
    def __init__(self,file_path):
        self.file_path = file_path
        self.x, self.y = self.csv_reader()
    
    def csv_reader(self):
        import csv
        x = []
        y = []
        with open(self.file_path,mode='r') as file:
            reader = csv.reader(file)
            next(reader)
            for row in reader:  
                x.append(torch.tensor(list(map(float,row[:-1])), dtype= torch.float32))
                if row[-1].lower() == "setosa":
                    y.append(torch.tensor(0,dtype=torch.long))  # Setosa

                elif row[-1].lower() == "versicolor":
                    y.append(torch.tensor(1,dtype=torch.long))

                elif row[-1].lower() == "virginica":
                    y.append(torch.tensor(2,dtype=torch.long))
                    
        x = torch.stack(x)
        y = torch.stack(y)
        return x, y

    
    def __len__(self):
        return len(self.x)

    def __getitem__(self,idx):
        return self.x[idx], self.y[idx]


dataset = CsvDataset(file_path='/kaggle/input/iris-dataset/iris.csv')

dataloader = DataLoader(dataset=dataset, batch_size= 1, num_workers= 4, shuffle= True)     
        

In [2]:
class IrisModel(nn.Module):
    def __init__(self,input_shape, hidden_shape, output_shape):
        super(IrisModel,self).__init__()
        self.l1 = nn.Linear(input_shape, hidden_shape)
        self.l2 = nn.Linear(hidden_shape, hidden_shape*2)
        self.l3 = nn.Linear(hidden_shape*2, output_shape)

    def forward(self, x):
        x = torch.relu(self.l1(x))
        x = torch.relu(self.l2(x))
        # x = torch.softmax(self.l3(x), dim=1)
        x = self.l3(x)
        return x

input_shape = 4
output_shape = 3


iris_model = IrisModel(input_shape= input_shape, hidden_shape= 16, output_shape= output_shape)


In [3]:
epochs = 100
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(iris_model.parameters(), lr=0.001)

for epoch in range(epochs):
    for feature, label in dataloader:
        optimizer.zero_grad()
        y_hat = iris_model(feature)
        error = loss_fn(y_hat, label)
        error.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}/{epochs} : {error.item():.4f}")
        

Epoch 10/100 : 0.6632
Epoch 20/100 : 0.5904
Epoch 30/100 : 0.0131
Epoch 40/100 : 0.1892
Epoch 50/100 : 0.0774
Epoch 60/100 : 0.0029
Epoch 70/100 : 0.0749
Epoch 80/100 : 0.0121
Epoch 90/100 : 0.0060
Epoch 100/100 : 0.0524


In [4]:
maping = {
    0 : "setosa",
    1 : "versicolor",
    2 : "virginica"
} 

with torch.no_grad():
    prediction = iris_model(torch.tensor([5.1, 3.5, 1.4, 0.2]))
    prediction = torch.argmax(prediction).item()
    print("Prediction: ",maping[prediction])

Prediction:  setosa
